In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

# "[공구] 일정 신청 및 관리 시트" 파일의 "공구전체일정" 탭을 csv로 다운로드하여 경로 추가
df = pd.read_csv('/Users/mac/Downloads/[공구] 일정 신청 및 관리 시트 - 공구전체일정 (3).csv')

# 친한스토어 어드민에서 최근 1년 주문내역 전체 엑셀 다운로드 및 경로 추가
fs_df = pd.read_excel('/Users/mac/Downloads/주문내역_2024-06-11~2025-06-11.xlsx')

# select * from pb_user where reg_dt > '2025-1-1'
# pb_user 테이블에 대해 위 쿼리를 돌리고 csv 다운로드하여 경로 추가
user_df = pd.read_csv('/Users/mac/Downloads/query_2-2025-06-11_112952.csv')

# select * from pb_user_auth_info where created_at > '2025-1-1'
# pb_user_auth_info 테이블에 대해 위 쿼리를 돌리고 csv 다운로드하여 경로 추가
auth_df = pd.read_csv('/Users/mac/Downloads/query_1-2025-06-11_94933.csv')

# select * from admin_group_buy_excel_order where order_ymd > '20250510'
# excel_group_buy_info 테이블에 대해 위 쿼리를 돌리되 결과값이 5만개 내로 마지막 날짜 조정
sales_df = pd.read_csv('/Users/mac/Downloads/query_3-2025-06-11_95113.csv')

# 친스 판매중, 판매종료된 공구 상품번호를 추가
gongu = [51414,51371,51367,51366,51365,51364,51363,51276,51275,51274,51273,51272,51271,51269,
         51268,51267,51260,51259,51258,51257,51256,51255,51254,51253,51182,51181,51178,51177,
         51176,51174,50557]

status = ['배송중', '배송완료', '구매확정', '배송준비', '결제완료']

fs_df = fs_df[fs_df['주문 상태'].isin(status)]
fs_gongu_df = fs_df[fs_df['상품 번호'].isin(gongu)]
fs_df = fs_df[~fs_df['상품 번호'].isin(gongu)]

today = datetime.today()

df.columns = df.loc[1]
df = df.drop([0,1])
con1 = df['진행상황-업체'] == '업체-일정확정'
con2 = df['진행상황-업체'] == '업체-공구종료'
df = df[con1 | con2]
df = df.reset_index()
df.loc[df['시작일'].str.len() <= 8, '시작일'] = '25/' + df['시작일']
df = df[df['시작일'].isna() == False]
df.drop(df[df['시작일'] == '약사님 일정 조율중'].index, inplace=True)

dates = []

for i in df['시작일']:
    date = i.split('(')[0].strip()
    year = '20' + date.split('/')[0].strip()
    month = date.split('/')[1].strip()
    day = date.split('/')[2].strip()
    final = year + '-' + month + '-' + day
    dates.append(final)

df['date'] = dates
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] >= datetime(2025, 2, 17)]

sales_df['time'] = pd.to_datetime(sales_df['reg_dt']) + timedelta(hours=9)
sales_df['minute'] = sales_df['time'].dt.strftime('%H:%M:%S')
sales_df['minute'] = pd.to_datetime(sales_df['minute'])
sales_df['minute'].mean()

datas = {}

for i in fs_gongu_df['상품명'].unique():
    con = fs_gongu_df['상품명'] == i
    value = fs_gongu_df.loc[con, '결제 완료일'].min()
    key = i
    datas[key] = value

temp_df = pd.DataFrame.from_dict(datas, orient="index", columns=['date'])

df_series = df.groupby(pd.Grouper(key='date', freq='W'))['진행상황-업체'].count()[5:20]
result_num1 = df_series[(df_series.keys() < today) & (df_series.keys() > today - timedelta(days=7))].values[0]
temp_df_series = temp_df.groupby(pd.Grouper(key='date', freq='W')).size()
result_num2 = temp_df_series[(temp_df_series.keys() < today) & (temp_df_series.keys() > today - timedelta(days=7))].values[0]
rate1 = result_num2 / (result_num1 + result_num2)
sales_df_series = sales_df.groupby(pd.Grouper(key='time', freq='W'))['order_price'].sum()
result_num3 = sales_df_series[(sales_df_series.keys() < today) & (sales_df_series.keys() > today - timedelta(days=7))].values[0]
fs_gongu_df_seires = fs_gongu_df.groupby(pd.Grouper(key='결제 완료일', freq='W'))['정산가'].sum()
result_num4 = fs_gongu_df_seires[(fs_gongu_df_seires.keys() < today) & (fs_gongu_df_seires.keys() > today - timedelta(days=7))].values[0]
rate2 = result_num4 / (result_num3 + result_num4)
count_series = sales_df.groupby(pd.Grouper(key='time', freq='W'))['order_no'].count()
result_num5 = count_series[(count_series.keys() < today) & (count_series.keys() > today - timedelta(days=7))].values[0]
fs_count_series = fs_gongu_df.groupby(pd.Grouper(key='결제 완료일', freq='W'))['결제 번호'].size()
result_num6 = fs_count_series[(fs_count_series.keys() < today) & (fs_count_series.keys() > today - timedelta(days=7))].values[0]
rate3 = result_num6 / (result_num5 + result_num6)
revenue_series = fs_df.groupby(pd.Grouper(key='결제 완료일', freq='W'))['정산가'].sum()
result_num7 = revenue_series[(revenue_series.keys() < today) & (revenue_series.keys() > today - timedelta(days=7))].values[0]
merged_df = user_df.merge(auth_df, left_on='id', right_on='user_id', how='inner')
merged_df['reg_dt'] = pd.to_datetime(merged_df['reg_dt'], format='mixed')
merged_df_series = merged_df.groupby(pd.Grouper(key='reg_dt', freq='W')).size()
result_num8 = merged_df_series[(merged_df_series.keys() < today) & (merged_df_series.keys() > today - timedelta(days=7))].values[0]
auth_df_di = auth_df[auth_df['di'].isna()]
auth_df_di['created_at'] = pd.to_datetime(auth_df_di['created_at'], format='mixed')
auth_df_di_series = auth_df_di.groupby(pd.Grouper(key='created_at', freq='W')).size()
result_num9 = auth_df_di_series[(auth_df_di_series.keys() < today) & (auth_df_di_series.keys() > today - timedelta(days=7))].values[0]

print(f'전체 공구 건수는 {result_num1 + result_num2}')
print(f'친한스토어 공구 건수는 {result_num2}')
print(f'친한스토어 공구 점유율은 {np.round(rate1 * 100, 2)}%\n')
print(f'전체 공구 매출은 {result_num3 + result_num4:,}원')
print(f'친스 공구 매출은 {result_num4:,}원')
print(f'친한스토어 매출 점유율은 {np.round(rate2 * 100, 2)}%\n')
print(f'전체 공구 판매건수는 {result_num5 + result_num6:,}건')
print(f'친스 공구 판매건수는 {result_num6:,}건')
print(f'친한스토어 판매건수 점유율은 {np.round(rate3 * 100, 2)}%\n')
print(f'친스 일반(친한구매) 매출은 {result_num7:,}원')
print(f'친스 매출 합계는 {result_num4 + result_num7:,}원\n')
print(f'전체 회원가입자 수는 {result_num8:,}명')
print(f'친스 회원가입자 수는 {result_num9:,}원')
print(f'친스 회원가입 점유율은 {np.round(result_num9 / result_num8 * 100, 2):,}%')

전체 공구 건수는 30
친한스토어 공구 건수는 2
친한스토어 공구 점유율은 6.67%

전체 공구 매출은 882,587,800원
친스 공구 매출은 38,655,500원
친한스토어 매출 점유율은 4.38%

전체 공구 판매건수는 7,961건
친스 공구 판매건수는 324건
친한스토어 판매건수 점유율은 4.07%

친스 일반(친한구매) 매출은 2,348,080원
친스 매출 합계는 41,003,580원

전체 회원가입자 수는 880명
친스 회원가입자 수는 503원
친스 회원가입 점유율은 57.16%
